In [1]:
import numpy as np
import h5py
import time
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
%matplotlib notebook
from collections import Counter
import math
import cv2
from numpy.polynomial import polynomial as P

In [2]:
file_paths = ["../predictions/20201119/bottomup/pred_FishTank20200416_160648/results/FishTank20200416_160648/FishTank20200416_160648.h5",
              "../predictions/20201119/bottomup/pred_FishTank20200413_154621/results/FishTank20200413_154621/FishTank20200413_154621.h5",
              "../predictions/20201120/pred_FishTank20200414_154100/results/FishTank20200414_154100/FishTank20200414_154100.h5",
              "../predictions/20201120/pred_FishTank20200415_154234/results/FishTank20200415_154234/FishTank20200415_154234.h5",
              "../predictions/20201120/pred_FishTank20200417_154139/results/FishTank20200417_154139/FishTank20200417_154139.h5",
              "../predictions/20201120/pred_FishTank20200418_160144/results/FishTank20200418_160144/FishTank20200418_160144.h5",
              "../predictions/20201120/pred_FishTank20200419_151352/results/FishTank20200419_151352/FishTank20200419_151352.h5",
              "../predictions/20201120/pred_FishTank20200419_173651/results/FishTank20200419_173651/FishTank20200419_173651.h5",
              "../predictions/20201120/pred_FishTank20200420_153056/results/FishTank20200420_153056/FishTank20200420_153056.h5",
              "../predictions/20201120/pred_FishTank20200420_175237/results/FishTank20200420_175237/FishTank20200420_175237.h5",
             ]

data_3D = []
for file_path in file_paths:
    with h5py.File(file_path, 'r') as f:
        data_3D.append(np.copy(f['tracks_3D']))

In [ ]:
for k, dataset in enumerate(data_3D):
    speeds = [[],[],[]]
    for bodypart in range(3):
        datapart = dataset[:, 0, bodypart, :]
        for i in np.arange(1, len(datapart)):
            if np.any(np.isnan(datapart[i])):
                speeds[bodypart].append(np.nan)
                continue

            j = i-1
            before = datapart[j]
            while np.any(np.isnan(before)):
                j -= 1
                before = datapart[j]
            
            difference = datapart[j] - datapart[i]
            speeds[bodypart].append(np.sqrt(sum(difference * difference)) * 100/(i-j))

    with h5py.File(file_paths[k], 'a') as hf:
        del hf['speed_frametoframe']
        hf.create_dataset('speed_frametoframe', data=speeds)

# Remove outliers

In [16]:
for k, dataset in enumerate(data_3D):
    data_test = np.copy(dataset)
    deleted = 0
    for bodypart in range(3):
        datapart = data_test[:, 0, bodypart, :]
        for i in np.arange(1, len(datapart)):
            if np.any(np.isnan(datapart[i])):
                continue

            j = i-1
            before = datapart[j]
            while np.any(np.isnan(before)):
                j -= 1
                before = datapart[j]
            
            difference = datapart[j] - datapart[i]
            if np.sqrt(sum(difference * difference)) * 100/(i-j) > 60:
                datapart[i] = [np.nan, np.nan, np.nan]
                deleted += 1

        data_test[:, 0, bodypart, :] = datapart
    print(deleted)
    
    with h5py.File(file_paths[k], 'a') as hf:
#         del hf['tracks_3D_no_outliers_60']
        hf.create_dataset('tracks_3D_no_outliers_60', data=data_test)

7564
6994
20268
2226
16007
8692
15592
13075
17685
15910


In [10]:
data_3D_filtered = []
for file_path in file_paths:
    with h5py.File(file_path, 'r') as f:
        data_3D_filtered.append(np.copy(f['tracks_3D_no_outliers_50']))

# Angles

In [ ]:
##### TAIL xy ######

def make_dot(v1, v2):
    dot = 0
    for i in range(len(v1)):
        dot += v1[i] * v2[i]
    return dot

def make_mag(vector):
    mag2 = 0
    for i in vector:
        mag2 += i**2
    mag = np.sqrt(mag2)
    return mag

def angle_between(v1, v2):
    dot = make_dot(v1, v2)
    mag1 = make_mag(v1)
    mag2 = make_mag(v2)
    angle = math.degrees(math.acos(dot/(mag1*mag2)))
    return angle

def tail_angles(data):
    head = data[:, 0, 0, :]
    thorax = data[:, 0, 1, :]
    tail = data[:, 0, 2, :]
    angles = []
    for i in range(len(head)):
        vector_head_flip = [(thorax[i][0] - head[i][0]), (thorax[i][1] - head[i][1])]
        vector_head = [(head[i][0] - thorax[i][0]), (head[i][1] - thorax[i][1])]
        vector_tail = [(tail[i][0] - thorax[i][0]), (tail[i][1] - thorax[i][1])]
        angle_smallest = angle_between(vector_head_flip, vector_tail)
        angle_tail = calc_angle_xpos(vector_tail)
        angle_head = calc_angle_xpos(vector_head)
        
        diff = angle_tail - angle_head
        if diff < 0:
            angle = 180 + diff
        else:
            angle = diff - 180
        angles.append(angle)
        
    return angles

########################

###### Upper body xy ######

def calc_angle_xpos(vector):
    return math.atan2(vector[1], vector[0]) * 180 / math.pi

def calculate_angles(data_1, data_2, axes):
    angles = []
    
    # A_vector = (x1 - x2)i + (y1 - y2)j
    for i in range(len(data_1)):
        x1 = data_1[i][axes[0]]
        x2 = data_2[i][axes[0]]
        y1 = data_1[i][axes[1]]
        y2 = data_2[i][axes[1]]
        
        angles.append(calc_angle_xpos([x1 - x2, y1 - y2]))
        
    return angles

def angle_directional(data_1, data_2, axes):
    angles = []
    for i in range(len(data_1)):
        x1 = data_1[i][axes[0]]
        x2 = data_2[i][axes[0]]
        y1 = data_1[i][axes[1]]
        y2 = data_2[i][axes[1]]
        angle = np.arctan2(y1 - y2, x1 - x2) * 180 / np.pi
        if angle < 0:
            angle = 360 + angle
        angles.append(angle)
    return angles

def angle_add_xy(data):
    angles = []
    pref_vector = [1, 0]
    for i in range(len(data)):
        x1 = data[i][0][0][0]
        x2 = data[i][0][1][0]
        y1 = data[i][0][0][1]
        y2 = data[i][0][1][1]
        vector = [x1-x2,y1-y2]
        
        angle_1 = calc_angle_xpos(vector)
        angle_2 = calc_angle_xpos(pref_vector)
        
        angle = np.mod(angle_1 - angle_2, 360)
        
        if angle > 180:
            angle = angle - 360
        
        if np.isnan(angle):
            angles.append(angle)
        elif len(angles) > 0:
            angles.append(pref_angle + angle)
            pref_angle = pref_angle + angle
        else:
            angles.append(angle)
            pref_angle = angle
        
        pref_vector = vector
        
    return angles

######################

######### z ##########

def z_xyplane_angle(data_1, data_2):
    angles = []
    for i in range(len(data_1)):
        angle = angle_between(data_1[i] - data_2[i], [data_1[i][0]-data_2[i][0], data_1[i][1]-data_2[i][1], 0])
        
        if data_1[i][2]-data_2[i][2] > 0:
            angles.append(angle)
        else: 
            angles.append(-angle)
    return angles

In [14]:
for k, dataset in enumerate(data_3D_filtered):
    
    angles_upperbody_180 = calculate_angles(dataset[:, 0, 0, :], dataset[:, 0, 1, :], [0,1])
    angles_upperbody_additive = angle_add_xy(dataset)
    angles_upperbody_directional = angle_directional(dataset[:, 0, 0, :], dataset[:, 0, 1, :], [0,1])
    z_angles_upperbody = z_xyplane_angle(dataset[:, 0, 0, :], dataset[:, 0, 1, :])
    tail_angles_xy = tail_angles(dataset)
    z_angles_tail = z_xyplane_angle(dataset[:, 0, 2, :], dataset[:, 0, 1, :])
    
    with h5py.File(file_paths[k], 'a') as hf:
        
#             del hf['upperbody_xy_angle_0to360']
            hf.create_dataset('upperbody_xy_angle_0to360', data=angles_upperbody_directional)
            
#             del hf['upperbody_xy_angle_180']
            hf.create_dataset('upperbody_xy_angle_180', data=angles_upperbody_180)
            
#             del hf['upperbody_xy_angle_additive']
            hf.create_dataset('upperbody_xy_angle_additive', data=angles_upperbody_additive)
            
#             del hf['upperbody_z_angle']
            hf.create_dataset('upperbody_z_angle', data=z_angles_upperbody)
            
#             del hf['tail_xy_angle']
            hf.create_dataset('tail_xy_angle', data=tail_angles_xy)
            
#             del hf['tail_z_angle']
            hf.create_dataset('tail_z_angle', data=z_angles_tail)

In [17]:
upperbody_angles_xy = []
upperbody_angles_z = []
tail_angles_xy = []
tail_angles_z = []

for file_path in file_paths:
    with h5py.File(file_path, 'r') as f:
        upperbody_angles_xy.append(np.copy(f['upperbody_xy_angle_additive']))
        upperbody_angles_z.append(np.copy(f['upperbody_z_angle']))
        tail_angles_xy.append(np.copy(f['tail_xy_angle']))
        tail_angles_z.append(np.copy(f['tail_z_angle']))

# Speed

In [ ]:
def derivative_poly(c, x):
    derivative = 0
    for i in range(1, len(c)):
        derivative += c[i] * (i) * np.power(x, i-1) 
    return derivative

def velocity_single_direction(data_i, df, degrees):
    
    velocities = []
    for i in range(df, len(data_i) - df + 1):
        data_r = data_i[i-df:i+df+1]
        c = P.polyfit(np.linspace(i - df, i - df + len(data_r) - 1, len(data_r)), data_r, degrees)
        
        if i == df:
            for j in range(0, df):
                velocities.append(derivative_poly(c, j) * 100)
                
        velocities.append(derivative_poly(c, i) * 100)
    
    for j in range(i+1, i+df):
        velocities.append(derivative_poly(c, j) * 100)
    
    return velocities

In [31]:
for k, dataset in enumerate(data_3D_filtered):
    
    t0 = time.time()
    
    speeds = np.zeros(shape=(len(dataset), 3, 3))
    for i in range(3):
        speeds[:, i, 0] = velocity_single_direction(dataset[:, 0, i, 0], 5, 2)
        speeds[:, i, 1] = velocity_single_direction(dataset[:, 0, i, 1], 5, 2)
        speeds[:, i, 2] = velocity_single_direction(dataset[:, 0, i, 2], 5, 2)

    with h5py.File(file_paths[k], 'a') as hf:
        #del hf['upperbody_xy_angle_0to360']
        hf.create_dataset('speed_xyz', data=speeds)
    
    tE = time.time()

    print(k, tE - t0)

0 585.6409499645233
1 558.6862783432007
2 628.8872640132904
3 644.4380295276642
4 601.5801794528961
5 584.2732632160187
6 567.9008369445801
7 566.9343507289886
8 562.1341927051544
9 578.8615207672119


In [33]:
for k in range(len(file_paths)):
    speed_upperbody_angles_xy = velocity_single_direction(upperbody_angles_xy[k], 5, 2)
    speed_upperbody_angles_z = velocity_single_direction(upperbody_angles_z[k], 5, 2)
    speed_tail_angles_xy = velocity_single_direction(tail_angles_xy[k], 5, 2)
    speed_tail_angles_z = velocity_single_direction(tail_angles_z[k], 5, 2)
    
    with h5py.File(file_paths[k], 'a') as hf:
        #del hf['upperbody_xy_angle_0to360']
        hf.create_dataset('speed_upperbody_angles_xy', data=speed_upperbody_angles_xy)
        #del hf['upperbody_xy_angle_0to360']
        hf.create_dataset('speed_upperbody_angles_z', data=speed_upperbody_angles_z)
        #del hf['upperbody_xy_angle_0to360']
        hf.create_dataset('speed_tail_angles_xy', data=speed_tail_angles_xy)
        #del hf['upperbody_xy_angle_0to360']
        hf.create_dataset('speed_tail_angles_z', data=speed_tail_angles_z)